<a href="https://colab.research.google.com/github/jumafernandez/clasificacion_correos/blob/main/notebooks/02-Generaci%C3%B3n_de_features_estaticas_sobre_Consulta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generación de features estáticas adicionales sobre el atributo Consulta

1. Levanto el archivo con los atributos pre-procesados:

In [24]:
from os import path
import pandas as pd

# En caso que no esté el archivo en Colab lo traigo
if not(path.exists("02-Correos_1er_preprocesamiento.csv")):
  !wget https://raw.githubusercontent.com/jumafernandez/clasificacion_correos/main/data/02-Correos_1er_preprocesamiento.csv

df = pd.read_csv('02-Correos_1er_preprocesamiento.csv', delimiter="|")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Consulta             1000 non-null   object
 1   dia_semana           1000 non-null   int64 
 2   semana_del_mes       1000 non-null   int64 
 3   mes                  1000 non-null   int64 
 4   cuatrimestre         1000 non-null   int64 
 5   anio                 1000 non-null   int64 
 6   hora_discretizada    1000 non-null   int64 
 7   dni_discretizado     1000 non-null   int64 
 8   legajo_discretizado  1000 non-null   int64 
 9   posee_legajo         1000 non-null   int64 
 10  posee_telefono       1000 non-null   int64 
 11  carrera_valor        1000 non-null   int64 
 12  proveedor_correo     1000 non-null   object
 13  Clase                1000 non-null   object
dtypes: int64(11), object(3)
memory usage: 109.5+ KB


2. Verifico cantidad de instancias y features:

In [25]:
df.shape

(1000, 14)

3. Genero las features estáticas léxicas basadas en caracteres:

In [26]:
# Total number of characters
def cantidad_caracteres(columna_consulta):
  return columna_consulta.str.len()

df['cantidad_caracteres'] = cantidad_caracteres(df.Consulta)

# Proporcion de mayúsculas en la consulta
def proporcion_mayusculas(consulta):
  return sum(1 for letra in consulta if letra.isupper())

df['proporcion_mayusculas'] = df['Consulta'].apply(proporcion_mayusculas)/df['cantidad_caracteres']

# Proporción de letras en la consulta
def proporcion_letras(consulta):
  return sum(1 for letra in consulta if letra.isalpha())

df['proporcion_letras'] = df['Consulta'].apply(proporcion_letras)/df['cantidad_caracteres']

# Cantidad de letras con tildes
def cantidad_tildes(consulta):
  return sum(1 for letra in consulta if letra.lower() in ['á', 'é', 'í', 'ó', 'ú'])

df['cantidad_tildes'] = df['Consulta'].apply(cantidad_tildes)

4. Genero las features estáticas léxicas basadas en palabras:

In [27]:
# total number of words
def cantidad_palabras(consulta):
  palabras = consulta.split(sep=' ')
  return len(palabras)

df['cantidad_palabras'] = df['Consulta'].apply(cantidad_palabras)

# proportion of short words (less than four characters)
def cantidad_palabras_cortas(consulta, letras_corta=4):
  palabras = consulta.split(sep=' ')
  return sum(1 for palabra in palabras if len(palabra) <= letras_corta)

df['cantidad_palabras_cortas'] = df['Consulta'].apply(cantidad_palabras_cortas)

# ratio of number of distinct words to the total number of words: |set(words)|/|words|
def cantidad_palabras_distintas(consulta, letras_corta=4):
  palabras = consulta.split(sep=' ')
  return len(set(palabras))

df['proporcion_palabras_distintas'] = df['Consulta'].apply(cantidad_palabras_distintas)/df['cantidad_palabras']
df['proporcion_palabras_distintas']

0      0.857143
1      0.916667
2      0.794118
3      0.969697
4      0.866667
         ...   
995    0.958333
996    0.925926
997    0.857143
998    0.920000
999    1.000000
Name: proporcion_palabras_distintas, Length: 1000, dtype: float64

5. Genero las features estáticas sintácticas:

In [28]:
# Frecuencia de signos de puntuación {,.¿?!:;’"}
def cantidad_signos_puntuacion(consulta):
  signos_puntuacion = [',', '.', '¿', '?', '!', '¡', ':', ';', '"']
  return sum(1 for letra in consulta if letra.lower() in signos_puntuacion)

df['frecuencia_signos_puntuacion'] = df['Consulta'].apply(cantidad_signos_puntuacion)/df['cantidad_caracteres']


6. Genero las features estáticas estructurales:

In [29]:
# total number of sentences
def cantidad_oraciones(consulta):
  if consulta.count('.')==0:
    return 1
  else: 
    return consulta.count('.')

df['cantidad_oraciones'] = df['Consulta'].apply(cantidad_oraciones)

# utiliza código de asignatura?
def utiliza_codigo_asignatura(consulta):
  codigo=0
  palabras = consulta.split(sep=' ')
  for palabra in palabras:
    if palabra.isdigit():
      if int(palabra)>=10000 and int(palabra)<=99999:
        codigo=1
  return codigo

df['utiliza_codigo_asignatura'] = df['Consulta'].apply(utiliza_codigo_asignatura)

6. Reordeno la clase para que quede última:

In [30]:
y = df['Clase']
df.drop(columns=['Clase'], inplace=True)
df.insert(13, "Clase", y, True)

In [31]:
df.head()

,Consulta,dia_semana,semana_del_mes,mes,cuatrimestre,anio,hora_discretizada,dni_discretizado,legajo_discretizado,posee_legajo,posee_telefono,carrera_valor,proveedor_correo,Clase,cantidad_caracteres,proporcion_mayusculas,proporcion_letras,cantidad_tildes,cantidad_palabras,cantidad_palabras_cortas,proporcion_palabras_distintas,frecuencia_signos_puntuacion,cantidad_oraciones,utiliza_codigo_asignatura
0,"hola quiero anotarme a las materias ,para el s...",0,1,8,2,2019,0,3,4,1,1,52,gmail,Vacunas Enfermería,162,0.0,0.802469,0,28,12,0.857143,0.030864,1,0
1,hola buenos días! quería saber cuando voy a po...,1,2,8,2,2017,1,3,3,1,1,52,outlook,Vacunas Enfermería,273,0.0,0.809524,5,48,26,0.916667,0.018315,3,0
2,hola quisiera saber si en la consulta de situa...,2,5,5,1,2017,3,8,3,1,0,54,gmail,Situación Académica,180,0.0,0.811111,5,34,19,0.794118,0.005556,1,0
3,buenas noches. en mi situacion academica apare...,0,1,2,1,2018,3,5,3,1,1,5,hotmail,Situación Académica,185,0.0,0.789189,1,33,16,0.969697,0.027027,5,0
4,"hola, quisiera obtener mi promedio o saber co...",5,1,8,2,2016,1,3,1,1,1,54,hotmail,Situación Académica,78,0.0,0.782051,0,15,7,0.866667,0.038462,2,0


7. Guardo el csv pre-procesado:

In [32]:
from google.colab import drive
drive.mount('drive')

df.to_csv('03-Correos_variables_estaticas.csv', index=False, sep='|')

!cp 03-Correos_variables_estaticas.csv "drive/My Drive/"

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).
